### Emoji prediction with GloVe embeddings and LSTM



In [2]:
import numpy as np
import pandas as pd
import emoji as emoji

In [3]:
print(emoji.EMOJI_DATA)
print(len(emoji.EMOJI_DATA))

{'🥇': {'en': ':1st_place_medal:', 'status': 2, 'E': 3}, '🥈': {'en': ':2nd_place_medal:', 'status': 2, 'E': 3}, '🥉': {'en': ':3rd_place_medal:', 'status': 2, 'E': 3}, '🆎': {'en': ':AB_button_(blood_type):', 'status': 2, 'E': 0.6, 'alias': [':ab:', ':ab_button_blood_type:']}, '🏧': {'en': ':ATM_sign:', 'status': 2, 'E': 0.6, 'alias': [':atm:', ':atm_sign:']}, '🅰️': {'en': ':A_button_(blood_type):', 'status': 2, 'E': 0.6, 'alias': [':a:', ':a_button_blood_type:'], 'variant': True}, '🅰': {'en': ':A_button_(blood_type):', 'status': 4, 'E': 0.6, 'alias': [':a:', ':a_button_blood_type:'], 'variant': True}, '🇦🇫': {'en': ':Afghanistan:', 'status': 2, 'E': 2, 'alias': [':flag_for_Afghanistan:', ':afghanistan:']}, '🇦🇱': {'en': ':Albania:', 'status': 2, 'E': 2, 'alias': [':flag_for_Albania:', ':albania:']}, '🇩🇿': {'en': ':Algeria:', 'status': 2, 'E': 2, 'alias': [':flag_for_Algeria:', ':algeria:']}, '🇦🇸': {'en': ':American_Samoa:', 'status': 2, 'E': 2, 'alias': [':flag_for_American_Samoa:', ':ameri

In [4]:
#We will only use 5 emojis for now
emoji_dictionary = {"0": "\u2764\uFE0F",
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }

for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [5]:
train = pd.read_csv('datasets/train_emoji.csv',header=None)
test = pd.read_csv('datasets/test_emoji.csv',header=None)

data = train.values
for i in range(10):
    print(data[i][0],emoji.emojize(emoji_dictionary[str(data[i][1])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😓
congratulations on your acceptance 😁
The assignment is too long  😓
I want to go play ⚾


In [6]:
#Load the glove embedding

embeddings = {}
with open('datasets/glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        embeddings[word] = coeffs

print(embeddings['eat'])
print(embeddings['play'])

[ 6.4295e-01 -4.2946e-01 -5.4277e-01 -1.0307e+00  1.2056e+00 -2.7174e-01
 -6.3561e-01 -1.5065e-02  3.7856e-01  4.6474e-02 -1.3102e-01  6.0500e-01
  1.6391e+00  2.3940e-01  1.2128e+00  8.3178e-01  7.3893e-01  1.5200e-01
 -1.4175e-01 -8.8384e-01  2.0829e-02 -3.2545e-01  1.8035e+00  1.0045e+00
  5.8484e-01 -6.2031e-01 -4.3296e-01  2.3562e-01  1.3027e+00 -8.1264e-01
  2.3158e+00  1.1030e+00 -6.0608e-01  1.0101e+00 -2.2426e-01  1.8908e-02
 -1.0931e-01  3.8350e-01  7.7362e-01 -8.1927e-02 -3.4040e-01 -1.5143e-03
 -5.6640e-02  8.7359e-01  1.4805e+00  6.9421e-01 -3.0966e-01 -9.0826e-01
  3.7277e-03  8.4550e-01]
[-0.73571   0.19937  -0.89408   0.36406  -0.20246  -0.034324 -0.63138
  0.76669  -0.94343   0.65883   0.049478  0.55608  -1.2809    0.44575
  0.73791   0.014728  0.80956  -0.35516  -1.0248   -0.13845  -0.47632
  0.32001   0.35023   0.77794   0.60233  -1.2321    0.043144 -0.41347
  0.34533  -1.3093    3.4681    0.9882    0.038253 -0.33672   0.30999
  0.6331    0.30798   0.68528  -0.21989 

In [7]:
#Create train and test datasets

from tensorflow.keras.utils import to_categorical

x_train,x_test = train[0], test[0]

y_train,y_test = to_categorical(train[1]), to_categorical(test[1])

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(132,)
(56,)
(132, 5)
(56, 5)


In [8]:
#Covert sentences into vectors

def getOutputEmbeddings(x):

    embedding_matrix_output = np.zeros((x.shape[0],10,50)) #(no. of sents in ds, max words in each sent, 50-D vector for each word)
    for ix in range(x.shape[0]):
        x[ix] = x[ix].split()
        for jx in range(len(x[ix])):
            embedding_matrix_output[ix][jx] = embeddings[x[ix][jx].lower()]
    return embedding_matrix_output

In [9]:
emb_Xtrain = getOutputEmbeddings(x_train)
emb_Xtest = getOutputEmbeddings(x_test)

/var/folders/33/d33y821d0gx3bx3vln89pxfc0000gn/T/ipykernel_6153/1236330150.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[ix] = x[ix].split()
/var/folders/33/d33y821d0gx3bx3vln89pxfc0000gn/T/ipykernel_6153/1236330150.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[ix] = x[ix].split()
/var/folders/33/d33y821d0gx3bx3vln89pxfc0000gn/T/ipykernel_6153/1236330150.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[ix] = x[ix].

In [10]:
print(emb_Xtrain.shape)
print(emb_Xtest.shape)

(132, 10, 50)
(56, 10, 50)


In [11]:
print(emb_Xtrain[1].shape) #10 vectors of size 50

(10, 50)


In [13]:
#two layer LSTM model

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.4))
#model.add(LSTM(64,input_shape=(10,50)))
#model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,765 (116.27 KB)

 Trainable params: 29,765 (116.27 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
#Train the mode

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(emb_Xtrain,y_train,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40


2026-02-03 08:59:02.632830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 283ms/step - acc: 0.1949 - loss: 1.6234 - val_acc: 0.2857 - val_loss: 1.6199
Epoch 2/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - acc: 0.3559 - loss: 1.5636 - val_acc: 0.2143 - val_loss: 1.6375
Epoch 3/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - acc: 0.3644 - loss: 1.5514 - val_acc: 0.2143 - val_loss: 1.6459
Epoch 4/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - acc: 0.3475 - loss: 1.5129 - val_acc: 0.1429 - val_loss: 1.6587
Epoch 5/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - acc: 0.3983 - loss: 1.4779 - val_acc: 0.2143 - val_loss: 1.6549
Epoch 6/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - acc: 0.4153 - loss: 1.4638 - val_acc: 0.2143 - val_loss: 1.6563
Epoch 7/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - acc: 0.3729 - loss: 1.4403 - val_acc: 0.2143 - val_loss: 1.6368
Epoch 8/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - acc: 0.4237 - loss: 1.3902 - val_acc: 0.2143 - val_loss: 1.6073
Epoch 9/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - acc: 0.4237 - loss: 1.3403 - val_acc: 0

In [15]:
model.evaluate(emb_Xtest,y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - acc: 0.5714 - loss: 1.7252


[1.7252217531204224, 0.5714285969734192]

In [19]:
#Display predicted emojis

pred = model.predict(emb_Xtest)
for i in range(5):
    print(x_test[i])
    print(emoji.emojize(emoji_dictionary[str(np.argmax(y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(pred[i]))]))
    print('-'*50)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
['I', 'want', 'to', 'eat']
🍴
🍴
--------------------------------------------------
['he', 'did', 'not', 'answer']
😓
😓
--------------------------------------------------
['he', 'got', 'a', 'raise']
😁
😁
--------------------------------------------------
['she', 'got', 'me', 'a', 'present']
❤️
😓
--------------------------------------------------
['ha', 'ha', 'ha', 'it', 'was', 'so', 'funny']
😁
😁
--------------------------------------------------
